In [26]:
!pip install surprise

In [4]:
!pip install matplotlib_venn

  Created wheel for matplotlib-venn: filename=matplotlib_venn-0.11.6-py3-none-any.whl size=32067 sha256=9e84250fd1060c1b679377a2ea14c3e5bda2ef29d20e9c391e41ab68d06ec76b
  Stored in directory: c:\users\horifox\appdata\local\pip\cache\wheels\82\e4\64\dd790d424818bc2f59c11471a1eee5dc8cfcd3f8ee8c4812fa
Successfully built matplotlib-venn


In [119]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from surprise import SVD, Dataset, Reader
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
%matplotlib inline 
sns.set(style="ticks")

import warnings
warnings.filterwarnings('ignore')

In [121]:
ratings = pd.read_csv('BX-Book-Ratings.csv', 
                    error_bad_lines=False, delimiter=';', 
                    encoding = 'ISO-8859-1')

users = pd.read_csv('BX-Users.csv', 
                    error_bad_lines=False, delimiter=';', 
                    encoding = 'ISO-8859-1')

books = pd.read_csv('BX-Books.csv', 
                    error_bad_lines=False, delimiter=';', 
                    encoding = 'ISO-8859-1')

data = pd.merge(ratings, users, on='User-ID', how='inner')
data = pd.merge(data, books, on='ISBN', how='inner')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [122]:
data.shape

(1031136, 12)

In [123]:
data.head()

,User-ID,ISBN,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,"tyler, texas, usa",NaN,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,"cincinnati, ohio, usa",23.0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,"strafford, missouri, usa",34.0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,"st. charles county, missouri, usa",2.0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,"beaverton, oregon, usa",NaN,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [124]:
data.isnull().sum()

User-ID                     0
ISBN                        0
Book-Rating                 0
Location                    0
Age                    277835
Book-Title                  0
Book-Author                 1
Year-Of-Publication         0
Publisher                   2
Image-URL-S                 0
Image-URL-M                 0
Image-URL-L                 4
dtype: int64

In [135]:
data = data.dropna(axis=0, how='any')
data = data.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)

In [136]:
data.isnull().sum()

User-ID                0
ISBN                   0
Book-Rating            0
Location               0
Age                    0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
dtype: int64

In [137]:
title = data['Book-Title'].values
author = data['Book-Author'].values
publisher = data['Publisher'].values
ISBN = data['ISBN'].values
rating = data['Book-Rating'].values

In [138]:
%%time
tfidfv = TfidfVectorizer()
title_matrix = tfidfv.fit_transform(title)
author_matrix = tfidfv.fit_transform(author)
publisher_matrix = tfidfv.fit_transform(publisher)
ISBN_matrix = tfidfv.fit_transform(ISBN)

Wall time: 20.4 s


In [144]:
print(matrix[1])

  (0, 8)	0.7071067811865476
  (0, 55)	0.7071067811865476


In [151]:
# класс для формирования рекомендаций на основе метода ближайших соседей
class SimpleKNNRecommender:
    
    def __init__(self, X_matrix, X_1, X_2, X_3):
        """
        Входные параметры:
        X_matrix - обучающая выборка (матрица объект-признак)
        """
        #Сохраняем параметры в переменных объекта
        self._X_matrix = X_matrix
        self.df = pd.DataFrame(
            {'title': pd.Series(X_1, dtype='str'),
            'author': pd.Series(X_2, dtype='str'),
            'publisher': pd.Series(X_3, dtype='str'),
            'dist': pd.Series([], dtype='float')})
            
            
    def recommend_for_single_object(self, K: int, \
                X_matrix_object, cos_flag = True, manh_flag = False):
        """
        Метод формирования рекомендаций для одного объекта.
        Входные параметры:
        K - количество рекомендуемых соседей 
        X_matrix_object - строка матрицы объект-признак, соответствующая объекту
        cos_flag - флаг вычисления косинусного расстояния
        manh_flag - флаг вычисления манхэттэнского расстояния
        Возвращаемое значение: K найденных соседей
        """
        
        scale = 1000000
        # Вычисляем косинусную близость
        if cos_flag:
            dist = cosine_similarity(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=False)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] < scale]
        
        else:
            if manh_flag:
                dist = manhattan_distances(self._X_matrix, X_matrix_object)
            else:
                dist = euclidean_distances(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=True)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] > 0.0]            
        
        # Оставляем К первых рекомендаций
        res = res.head(K)
        return res

In [154]:
sparks_matrix = author_matrix[0]
skr1 = SimpleKNNRecommender(author_matrix, title, author, publisher)
rec1 = skr1.recommend_for_single_object(5, sparks_matrix)
rec1

,title,author,publisher,dist
645492,"Goodbye, Good Men: How Liberals Brought Corrup...",Michael S. Rose,Regnery Publishing,846787.929526
722597,Regions of the Heart : The Triumph and Tragedy...,David Rose,National Geographic,841000.601267
626596,Thief of Olympus,Elizabeth Rose,Wigwam Publishing Co.,819570.484541
590247,Freeing the Whales: How the Media Created the ...,Tom Rose,Carol Publishing Corporation,807830.167057
696252,101 Ways to Date Your Mate,Rose Smith,William Havens Publishing Company,800360.034775


In [157]:
# При поиске с помощью Евклидова расстояния 
rec2 = skr1.recommend_for_single_object(5, sparks_matrix, cos_flag = False)
rec2

,title,author,publisher,dist
645492,"Goodbye, Good Men: How Liberals Brought Corrup...",Michael S. Rose,Regnery Publishing,553555.905892
722597,Regions of the Heart : The Triumph and Tragedy...,David Rose,National Geographic,563913.820957
626596,Thief of Olympus,Elizabeth Rose,Wigwam Publishing Co.,600715.432562
590247,Freeing the Whales: How the Media Created the ...,Tom Rose,Carol Publishing Corporation,619951.341547
696252,101 Ways to Date Your Mate,Rose Smith,William Havens Publishing Company,631886.010646


In [158]:
# Манхэттэнское расстояние
rec3 = skr1.recommend_for_single_object(5, sparks_matrix, cos_flag = False, manh_flag = True)
rec3

,title,author,publisher,dist
645492,"Goodbye, Good Men: How Liberals Brought Corrup...",Michael S. Rose,Regnery Publishing,685142.707313
722597,Regions of the Heart : The Triumph and Tragedy...,David Rose,National Geographic,700033.583114
626596,Thief of Olympus,Elizabeth Rose,Wigwam Publishing Co.,753407.892773
590247,Freeing the Whales: How the Media Created the ...,Tom Rose,Carol Publishing Corporation,781585.154420
696252,101 Ways to Date Your Mate,Rose Smith,William Havens Publishing Company,799159.618560
